# Tic Tac Toe

## Project imports

In [6]:
import ast
import os
import sys

sys.path.append(os.path.abspath(".."))

import pandas as pd

## Classification training

### Loading playable game states

In [7]:
game_file = "game_state_targets"
path = "../data/"
ext = ".csv"
headers = ["state", "last_player", "target"]

games = pd.read_csv(path + game_file + ext)
games[headers[0]] = games[headers[0]].apply(ast.literal_eval)
print(games.head())

                            state  last_player  target
0  [0, -1, 1, 1, 0, -1, -1, 1, 1]            1       0
1   [1, 0, 1, 0, 0, 0, -1, 0, -1]           -1       1
2  [0, 0, 0, -1, 1, 1, -1, 1, -1]           -1       1
3   [1, 0, 0, -1, -1, 1, 0, 0, 0]           -1       2
4  [0, 1, 1, -1, -1, 0, 1, -1, 0]           -1       0


### Training

- Setup

In [8]:
import torch
import torch.nn as nn

from models.neural_net import TicTacToeNet

net = TicTacToeNet()
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.01)
num_epochs = 1000
batch_size = 100

states = torch.tensor(games["state"].to_list(), dtype=torch.float32)
targets = torch.tensor(games["target"].values, dtype=torch.long)

- Training

In [9]:
for epoch in range(num_epochs):
    permutation = torch.randperm(states.size()[0])
    states = states[permutation]
    targets = targets[permutation]

    for i in range(0, states.size()[0], batch_size):
        batch_states = states[i : i + batch_size]
        batch_targets = targets[i : i + batch_size]

        outputs = net(batch_states)
        loss: torch.Tensor = loss_fn(outputs, batch_targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

torch.save(net.state_dict(), path + "models/mlp_model.pth")

Epoch 1/1000, Loss: 2.199054479598999
Epoch 1/1000, Loss: 2.184976100921631
Epoch 1/1000, Loss: 2.1801950931549072
Epoch 1/1000, Loss: 2.155712366104126
Epoch 1/1000, Loss: 2.137887477874756
Epoch 1/1000, Loss: 2.1333446502685547
Epoch 1/1000, Loss: 2.096027374267578
Epoch 1/1000, Loss: 2.167431354522705
Epoch 1/1000, Loss: 2.1298584938049316
Epoch 1/1000, Loss: 2.13446044921875
Epoch 1/1000, Loss: 2.1481072902679443
Epoch 1/1000, Loss: 2.094914436340332
Epoch 1/1000, Loss: 2.1449756622314453
Epoch 1/1000, Loss: 2.1170225143432617
Epoch 1/1000, Loss: 2.119312047958374
Epoch 1/1000, Loss: 2.1359739303588867
Epoch 1/1000, Loss: 2.110602378845215
Epoch 1/1000, Loss: 2.0640478134155273
Epoch 1/1000, Loss: 2.089440107345581
Epoch 1/1000, Loss: 2.0318193435668945
Epoch 1/1000, Loss: 2.0358998775482178
Epoch 1/1000, Loss: 2.0350725650787354
Epoch 1/1000, Loss: 2.1136858463287354
Epoch 1/1000, Loss: 2.038947582244873
Epoch 1/1000, Loss: 2.037658452987671
Epoch 1/1000, Loss: 2.003998279571533
E

### Testing

In [10]:
net.eval()
outputs = net(states)

_, predicted = torch.max(outputs, 1)

accuracy = (predicted == targets).sum().item() / targets.size(0)
print(f"Test Accuracy: {round(accuracy * 100, 2)}%")

Test Accuracy: 90.58%
